# IS 430 Final Project - Workout Generator

By: Mitchell Goeringer

## Use this Generator to Craft the Perfect Workout!

In [1]:
import pandas as pd # Imports the modules that will be used in this program
from IPython.display import display, clear_output, HTML
import ipywidgets as w

df = pd.read_csv("WorkoutDataset.csv", encoding = "unicode-escape") # Reads in the dataset

lower_case = {c.lower(): c for c in df.columns}

# Creates identifiers for the columns, in lowercase to avoid possible issues with capitalization
name_col = lower_case.get("title")
desc_col = lower_case.get("desc")
bp_col = lower_case.get("bodypart")
eq_col = lower_case.get("equipment")
lvl_col = lower_case.get("level")

# Creates options in alphabetical order, except the difficulty levels, which are sorted by their respective levels
def opts(col):
    a = df.get(col).dropna()
    return sorted(b for b in a.unique() if b)

bp_opts = opts(bp_col) # Determines the options that can be selected in the dropdowns in the widget
eq_opts = opts(eq_col)
lv_opts = ["Beginner","Intermediate","Expert"]

# Sets up the widgets to filter the differet workouts based on desired bodypart, equipment available, and difficulty level
bp_dis = w.Dropdown(options = bp_opts, description = "Bodypart:")
eq_dis = w.Dropdown(options = (["Any"] + eq_opts), description = "Equipment:")
lv_dis = w.Dropdown(options = lv_opts, description = "Level:")
but = w.Button(description = "Display Workouts!", button_style = "info")
out = w.Output()

# Removed the numerical values assigned to each exercise by the dataset and allowed it to be displayed in a table
def table(df_view):
        display(HTML(df_view.to_html(index=False)))

# Makes a copy of the dataframe in order to sort through and avoid complications in the selection process
def view(bp, eq, lv, n=10):
    new_df = df.copy()
    if bp and bp_col: new_df = new_df[new_df[bp_col].str.lower()==bp.lower()]
    if eq and eq != "Any": new_df = new_df[new_df[eq_col].str.contains(eq, case=False, na=False)]
    if lv: new_df = new_df[new_df[lvl_col].str.lower()==lv.lower()]

    if new_df.empty: # Displays an error message if there are no exact matches
        print("No Exact Matches :( Showing top results for selected bodypart:")
        new_df = df[df[bp_col].str.lower()==bp.lower()]

    if desc_col in new_df.columns: # Puts workouts with a description recomended more often
        new_df = new_df.sort_values(by=desc_col)

# Sets the number of sets and reps based on the difficulty level, as well as allows for the sets to be changed if an exact match is not found
    new_df["Sets"] = 3 if "Expert" in lv else 2 if "Intermediate" in lv else 1
    new_df["Reps"] = 10

    columns = [] # Orders the table so that descriptions come last, and adds in the sets and reps columns
    if name_col: columns.append(name_col)
    if bp_col: columns.append(bp_col)
    if eq_col: columns.append(eq_col)
    if lvl_col: columns.append(lvl_col)
    columns = columns + ["Sets","Reps"]
    if desc_col: columns.append(desc_col)
    new_df = new_df[columns].head(n)

    rename = {} # Renames the columns for a better user experience as well as formating the table
    rename[name_col] = "Exercise"
    rename[bp_col] = "Bodypart"
    rename[eq_col] = "Equipment Needed"
    rename[lvl_col] = "Difficulty Level"
    rename[desc_col] = "Description"
    table(new_df.rename(columns=rename))

def click(_): # Controls what is displayed once the button is clicked
    with out:
        clear_output()
        view(bp_dis.value, eq_dis.value, lv_dis.value, n=10) # Decides the number of exercises displayed in the output

but.on_click(click)
display(w.VBox([bp_dis, eq_dis, lv_dis, but, out])) # Final display where the selections are made